_Control

In [ ]:
%reset -f

In [ ]:
# import
import os
import numpy as np
import sympy as sp
import matplotlib as mpl
import pandas as pd
import jax
import jax.numpy as jnp

dspath="./datasets"

In [ ]:
def dummyf(x):
	return x**2

_Notes

In [ ]:
# matplotlib
import matplotlib.pyplot as plt
import numpy as np

"""
for i in range(decoded.shape[0]):

    array=decoded[i,0,:,:,0]
    
    # Plot the array as a grayscale image
    plt.imshow(array, cmap='gray')

    plt.colorbar()  # Add a color bar to the side for reference
    plt.title('Grayscale Image of 128x128 Array')
    plt.show()
"""

fig, axes = plt.subplots(10, 6, figsize=(12, 15))
for i in range(10):
	for j in range(6):
		array=decoded[6*i+j,0,:,:,0]
		axes[i, j].imshow(array, cmap='gray')
		axes[i, j].set_title(f"Plot {6*i+j+1}")
plt.tight_layout()
plt.show()

In [ ]:
# jax checkpoints
import jax.numpy as jnp
from flax.training import checkpoints
import os
ckpt_dir = os.getcwd()+"/checkpoints/TEST"

restored={}
state = {'a':jnp.ones(16)}
#restored = checkpoints.restore_checkpoint(ckpt_dir, state, prefix=f"_ARRAY_")

checkpoints.save_checkpoint(ckpt_dir, state, 1, prefix=f"_ARRAY_", keep=5, overwrite=True)

In [ ]:
# basic TrainState
import flax.linen as nn
from flax.training.train_state import TrainState
import jax, jax.numpy as jnp
import optax

x = jnp.ones((1,2))
y = jnp.ones((1,2))
z = jnp.array([[1,1],[-2,4],[0,3]])
model = nn.Dense(2)
variables = model.init(jax.random.key(0), x)
tx = optax.adam(2e-2)

state=TrainState.create(apply_fn=model.apply, params=variables, tx=tx)

#print(state.params['params']['kernel'])
#print(model.apply(state.params,z))

def loss(params, x, y):
	predictions=state.apply_fn(params,x)
	return optax.l2_loss(predictions,y).mean()
print(loss(state.params,x,y))

for _ in range(300):
	
	grads=jax.grad(loss)(state.params,x,y)
	state=state.apply_gradients(grads=grads)
	#display(loss(state.params,x,y))

print(state.params)
print(loss(state.params,x,y))

In [ ]:
# transformers pipeline
# long loading time
from transformers import pipeline

'''# Test a pre-trained pipeline (e.g., sentiment-analysis)
classifier = pipeline(task='sentiment-analysis',device='cuda')
print(classifier("Hello World"))
print(classifier(["Hugging Face is amazing!","Red Blue yellow"]))

generator = pipeline("text-generation", model="gpt2", device='cuda')
print(generator("Once upon a time", max_length=50, num_return_sequences=1))
print(generator("I've always been shy growing up.", max_length=250, num_return_sequences=1))'''


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens1 = tokenizer("Hello, how are you?", return_tensors="pt")
tokens2 = tokenizer("Hello", return_tensors="pt")
print(tokens1)
print(tokens2)

In [ ]:
# PyTorch BatchNorm evaluation mode issue
import torch
import torch.nn as nn
import numpy as np

# Define a BatchNorm layer
bn = nn.BatchNorm1d(3,momentum=0.1)

# Simulate training mode
bn.train()  # Training mode
input_train = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
output_train = bn(input_train)
print("Training Mode Output:", output_train)

'''mean_rows = np.mean(np.array(input_train), axis=0)

# Variance along columns (axis=0)
variance_cols = np.var(np.array(input_train), axis=0)

print("Mean along rows:", mean_rows)
print("Variance along columns:", variance_cols)'''


'''print([pn, p] for pn,p in bn.named_parameters())
for pn,p in bn.named_parameters():
	print([pn,p])'''
print(bn.state_dict())

# Switch to evaluation mode
bn.eval()  # Evaluation mode
input_eval = torch.tensor([[4.0, 1.0, 6.0],[7.0, 8.0, 9.0]])
output_eval = bn(input_eval)
print("Evaluation Mode Output:", output_eval)
print(bn.state_dict())

input_eval = torch.tensor([[7.0, 8.0, 9.0]])
output_eval = bn(input_eval)
print("Evaluation Mode Output:", output_eval)
print(bn.state_dict())

In [8]:
# torch .backward, checkpoint
from typing import Iterable
from networkx import non_randomness
from sqlalchemy import false
import torch
import torch.nn as nn
import numpy as np
from torch.utils.checkpoint import checkpoint


x = torch.tensor(4.0, requires_grad=True)
y = x ** 2  # y = x²
y.retain_grad()
z = torch.sin(y)

print(z)
print(y)
print(x)

z.backward()  # Compute gradients
print(x.grad)  # Output: 4.0 (dy/dx = 2*x, at x=2)
print(y.grad)
print(z.grad)

# Define a function to be checkpointed
def forward_func(input):
    return torch.relu(input)
#pdb.set_trace()

# Input tensor
input_tensor = torch.randn(5, requires_grad=True)

# Use checkpoint
output = checkpoint(forward_func, input_tensor, use_reentrant=False)

print(output)

# Backward pass
z=output.sum()
z.backward()
print(input_tensor.grad)

a1=torch.tensor(2.0, requires_grad=True)
a2=torch.tensor(1.0, requires_grad=True)
#a3=torch.tensor([a1+2*a2, a1*a2])
a3=a1+2*a2
a3.backward(gradient=torch.tensor(1))
#a3.backward()
print( [i.grad for i in iter([a1,a2,a3])] )



tensor(-0.2879, grad_fn=<SinBackward0>)
tensor(16., grad_fn=<PowBackward0>)
tensor(4., requires_grad=True)
tensor(-7.6613)
tensor(-0.9577)
None
tensor([0.0000, 0.7638, 0.0000, 0.0000, 1.4254], grad_fn=<ReluBackward0>)
tensor([0., 1., 0., 0., 1.])
[tensor(1.), tensor(2.), None]


/tmp/ipykernel_746837/1281635405.py:23: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(z.grad)
/tmp/ipykernel_746837/1281635405.py:49: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 fo

In [ ]:
# @wrap decorator
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Wrapper called")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_function():
    """This is my function."""
    pass

print(my_function.__doc__) 
my_function()
#my_decorator(my_function)()

In [ ]:
# ZhipuAI video content description
from zhipuai import ZhipuAI

client = ZhipuAI(api_key='c40bdf30c49ef1d2f40a98b10af2df84.qibzbV4iQBseorFg') # Fill in your own APIKey

'''response = client.chat.completions.create(
    model="glm-4v-plus",  # Fill in the model name to be called
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "video_url",
            "video_url": {
                "url" : "https://videos.pexels.com/video-files/3195394/3195394-sd_426_240_25fps.mp4"
            }
          },
          {
            "type": "text",
            "text": "Please describe this video in detail"
          }
        ]
      }
    ]
)'''

'''response = client.chat.completions.create(
    model="glm-4-plus",  # Fill in the model code you need to call
    messages=[
        {"role": "system", "content": "You are an assistant who loves to answer various questions, your task is to provide users with professional, accurate, and insightful advice."},
        {"role": "user", "content": "A farmer needs to take a wolf, a sheep, and cabbage across a river, but can only take one item at a time, and the wolf and sheep cannot be left alone, nor can the sheep and cabbage. How should the farmer cross the river?"}
    ],
)'''

response = client.chat.completions.create(
    model="glm-4-plus",  # Fill in the model code you need to call
    messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": "Evaluate the integral ∫x²dx"}
    ],
)
print(response.choices[0].message)

In [ ]:
#all-MiniLM-L6-v2, embedding, 23M params, usage;
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

In [ ]:
# weight must be cloned
n, d, m = 3, 5, 7
embedding = nn.Embedding(n, d, max_norm=1.0)
W = torch.randn((m, d), requires_grad=True)
idx = torch.tensor([1, 2])
a = embedding.weight.clone() @ W.t()  # weight must be cloned for this to be differentiable
b = embedding(idx) @ W.t()  # modifies weight in-place
out = (a.unsqueeze(0) + b.unsqueeze(1))
loss = out.sigmoid().prod()
loss.backward()

In [2]:
# window calculator
import tkinter as tk
from tkinter import messagebox

def calculate_sum():
    try:
        num1 = float(entry1.get())
        num2 = float(entry2.get())
        result = num1 + num2
        messagebox.showinfo("Result", f"The sum is: {result}")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers")

# Create the main window
window = tk.Tk()
window.title("Sum Calculator")

# Create and place the labels and entry widgets
tk.Label(window, text="Enter the first number:").grid(row=0, column=0)
entry1 = tk.Entry(window)
entry1.grid(row=0, column=1)

tk.Label(window, text="Enter the second number:").grid(row=1, column=0)
entry2 = tk.Entry(window)
entry2.grid(row=1, column=1)

# Create and place the calculate button
button = tk.Button(window, text="Calculate", command=calculate_sum)
button.grid(row=2, columnspan=2)

# Start the GUI event loop
window.mainloop()
